In [ ]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv
import igraph as ig
from pandas import Timestamp

In [2]:
def load_data(data_dir: str, output_dir: str) -> pd.DataFrame:
    """
    Load the dataset and reference files.

    This method loads the main DataFrame, cluster labels, and legend files
    required for the analysis. It ensures that all necessary data is available
    for subsequent operations.

    Returns:
        self: The ClusterAnalyzer instance with loaded data.
    """
    # Load DataFrame
    pdf = os.path.join(data_dir, "08-analysis-data/df_analysis.pkl")
    df = pd.read_pickle(pdf)
    print(f"DataFrame loaded with rows: {len(df)}")

    # Load graph
    pg = os.path.join(data_dir, "08-analysis-data/graph_analysis.graphml")
    g = ig.read(pg)
    print(f"Graph loaded with {g.vcount()} vertices and {g.ecount()} edges.")
    # Load cluster labels
    labels_path = os.path.join(
        output_dir,
        "cluster-qualifications_2025/cluster-label-tree/cluster_labels_filtered.json",
    )
    with open(labels_path, "r") as f:
        cluster_label_dict = json.load(f)
    print(f"Cluster labels loaded with {len(cluster_label_dict)} entries.")

    # Load legend
    legend_path = os.path.join(
        output_dir,
        "cluster-qualifications_2025/cluster-label-tree/legend_labels_2025.json",
    )
    with open(legend_path, "r") as f:
        legend = json.load(f)
    print(f"Legend loaded with {len(legend)} entries.")
    return df, g, cluster_label_dict, legend


load_dotenv()

# Access environment variables
data_dir = os.getenv("DATA_DIR")
output_dir = os.getenv("OUTPUT_DIR")

df, g, cluster_label_dict, legend = load_data(data_dir=data_dir, output_dir=output_dir)

DataFrame loaded with rows: 36510
Graph loaded with 36510 vertices and 551227 edges.
Cluster labels loaded with 99 entries.
Legend loaded with 4 entries.


/opt/homebrew/Caskroom/miniconda/base/envs/bibliometrics/lib/python3.11/site-packages/igraph/io/files.py:295: RuntimeWarning: Could not add vertex ids, there is already an 'id' vertex attribute. at src/io/graphml.c:488
  return reader(f, *args, **kwds)


In [57]:
g.es[10]

igraph.Edge(<igraph.Graph object at 0x1651d9e50>, 10, {'weight': 0.634322166442871})

In [58]:
g.vs[10]

igraph.Vertex(<igraph.Graph object at 0x1651d9e50>, 10, {'eid': '2-s2.0-0020144348', 'title': 'Serotonin and fear retention in the rat', 'year': 1982.0, 'id': '10', 'cluster': '47', 'centrality_alpha0.3_k10_res0.002': 0.0157127297097148, 'doi': '10.1037/h0077897', 'authors': 'Archer'})

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from typing import Dict, List, Tuple, Any


def calculate_modularity(snapshot_graph, cluster_attribute="cluster"):
    """
    Calculate modularity for a graph with cluster assignments.

    Parameters:
    -----------
    snapshot_graph : igraph.Graph
        The graph to analyze
    cluster_attribute : str
        The vertex attribute containing cluster assignments

    Returns:
    --------
    float : The modularity value
    """
    # Get all clusters
    clusters = set(snapshot_graph.vs[cluster_attribute])

    # Build cluster membership mapping
    cluster_nodes = defaultdict(list)
    for v in snapshot_graph.vs:
        cluster_nodes[v[cluster_attribute]].append(v.index)

    # Build edge matrix between clusters
    cluster_edge_matrix = defaultdict(lambda: defaultdict(int))

    # Count internal edges for each cluster
    internal_edges = defaultdict(int)

    for edge in snapshot_graph.es:
        source_cluster = snapshot_graph.vs[edge.source][cluster_attribute]
        target_cluster = snapshot_graph.vs[edge.target][cluster_attribute]

        if source_cluster == target_cluster:
            internal_edges[source_cluster] += 1
        else:
            cluster_edge_matrix[source_cluster][target_cluster] += 1
            cluster_edge_matrix[target_cluster][source_cluster] += 1

    # Calculate modularity
    total_edges = snapshot_graph.ecount()

    if total_edges == 0:
        return 0

    modularity = 0

    # Calculate degree for each cluster (sum of all edges touching nodes in cluster)
    cluster_degrees = defaultdict(int)
    for cluster in clusters:
        # Internal edges count twice for degree
        cluster_degrees[cluster] = internal_edges[cluster] * 2
        # Add external edges
        cluster_degrees[cluster] += sum(cluster_edge_matrix[cluster].values())

    # Calculate modularity using the standard formula
    for cluster in clusters:
        # Actual internal edges
        actual_internal = internal_edges[cluster]

        # Expected edges under null model
        expected_internal = (cluster_degrees[cluster] ** 2) / (4 * total_edges)

        modularity += actual_internal - expected_internal

    modularity = modularity / total_edges

    return modularity


def create_temporal_snapshots(graph, years, time_window_size=3):
    """
    Create temporal snapshots of the graph based on time windows.

    Parameters:
    -----------
    graph : igraph.Graph
        The full graph with 'year' vertex attribute
    years : list
        List of years to consider
    time_window_size : int
        Size of the time window in years

    Returns:
    --------
    dict : Dictionary with (start_year, end_year) tuples as keys
    """
    time_periods = range(min(years), max(years) + 1, time_window_size)
    temporal_data = {}

    for year_start in time_periods:
        year_end = year_start + time_window_size - 1

        # Identify nodes published within this time window
        nodes_in_window = [n for n in graph.vs if year_start <= n["year"] <= year_end]

        # Create snapshot graph
        snapshot_graph = graph.subgraph(nodes_in_window).copy()

        # Calculate modularity
        modularity = calculate_modularity(snapshot_graph)

        # Store results
        temporal_data[(year_start, year_end)] = {
            "graph": snapshot_graph,
            "n_nodes": snapshot_graph.vcount(),
            "n_edges": snapshot_graph.ecount(),
            "n_clusters": len(set(snapshot_graph.vs["cluster"])),
            "modularity": modularity,
        }

        print(f"\nPeriod {year_start}-{year_end}:")
        print(f"  Nodes: {snapshot_graph.vcount()}, Edges: {snapshot_graph.ecount()}")
        print(f"  Clusters: {len(set(snapshot_graph.vs['cluster']))}")
        print(f"  Modularity: {modularity:.4f}")

    return temporal_data


def extract_time_series(temporal_data):
    """
    Extract time series data for plotting.

    Parameters:
    -----------
    temporal_data : dict
        Dictionary returned by create_temporal_snapshots

    Returns:
    --------
    dict : Dictionary with metric names as keys and lists of values
    """
    time_series = {
        "periods": [],
        "modularity": [],
        "n_nodes": [],
        "n_edges": [],
        "n_clusters": [],
    }

    for (year_start, year_end), data in sorted(temporal_data.items()):
        time_series["periods"].append(f"{year_start}-{year_end}")
        time_series["modularity"].append(data["modularity"])
        time_series["n_nodes"].append(data["n_nodes"])
        time_series["n_edges"].append(data["n_edges"])
        time_series["n_clusters"].append(data["n_clusters"])

    return time_series


# Main analysis function
def analyze_temporal_modularity(
    graph, years=None, time_window_size=3, plot=True, save_plot=None
):
    """
    Perform complete temporal modularity analysis.

    Parameters:
    -----------
    graph : igraph.Graph
        The graph to analyze (must have 'year' and 'cluster' vertex attributes)
    years : list, optional
        List of years to consider (if None, extracts from graph)
    time_window_size : int
        Size of time windows in years
    plot : bool
        Whether to create plots
    save_plot : str, optional
        Path to save the plot

    Returns:
    --------
    tuple : (temporal_data, time_series)
    """
    # Extract years if not provided
    if years is None:
        years = list(range(min(graph.vs["year"]), max(graph.vs["year"]) + 1))

    print(f"Analyzing years: {min(years)} to {max(years)}")
    print(f"Time window size: {time_window_size} years")

    # Create temporal snapshots
    temporal_data = create_temporal_snapshots(graph, years, time_window_size)

    # Extract time series
    time_series = extract_time_series(temporal_data)

    return temporal_data, time_series

In [8]:
years = list(range(1982, 2026))
temporal_data, time_series = analyze_temporal_modularity(g, years, time_window_size=3)

Analyzing years: 1982 to 2025
Time window size: 3 years

Period 1982-1984:
  Nodes: 235, Edges: 274
  Clusters: 44
  Modularity: 0.5711

Period 1985-1987:
  Nodes: 331, Edges: 448
  Clusters: 51
  Modularity: 0.5578

Period 1988-1990:
  Nodes: 514, Edges: 633
  Clusters: 62
  Modularity: 0.6769

Period 1991-1993:
  Nodes: 1045, Edges: 1865
  Clusters: 85
  Modularity: 0.6021

Period 1994-1996:
  Nodes: 1606, Edges: 3237
  Clusters: 94
  Modularity: 0.5975

Period 1997-1999:
  Nodes: 2217, Edges: 4705
  Clusters: 96
  Modularity: 0.6138

Period 2000-2002:
  Nodes: 2547, Edges: 4965
  Clusters: 98
  Modularity: 0.6317

Period 2003-2005:
  Nodes: 2914, Edges: 5410
  Clusters: 98
  Modularity: 0.6631

Period 2006-2008:
  Nodes: 3449, Edges: 7008
  Clusters: 99
  Modularity: 0.6539

Period 2009-2011:
  Nodes: 3594, Edges: 6418
  Clusters: 99
  Modularity: 0.6794

Period 2012-2014:
  Nodes: 3841, Edges: 6375
  Clusters: 99
  Modularity: 0.6851

Period 2015-2017:
  Nodes: 4024, Edges: 7207
  